# RUN IN JULIA

In [1]:
# import Pkg
# Pkg.add("HDF5")

In [2]:
using JuMP
using Gurobi
using CSV
using DataFrames
using HDF5


In [3]:
Tx = CSV.read("inputs/Tx.csv", DataFrame, drop=1:1) |> Matrix #N, N
Ty = CSV.read("inputs/Ty.csv",DataFrame, drop=1:1)|> Matrix; #M, N
Tz = CSV.read("inputs/Tz.csv",DataFrame, drop=1:1)|> Matrix; #N, M

M,N = size(Ty)
Cf = 0.1
Cw = 0
Ct = 0
alpha = 0.1
println("M: ", M)
println("N: ", N)

M: 95
N: 5


In [ ]:
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "TimeLimit", 120)

@variable(model, o[1:M], Bin)
@variable(model, x[1:N, 1:N, 1:N], Bin)
@variable(model, y[1:N, 1:M, 1:N], Bin)
@variable(model, z[1:N, 1:N, 1:M], Bin)
@variable(model, f[1:N] >= 0)
@variable(model, u[1:N, 1:N]) #k, j
@variable(model, L >= 0)

@objective(model, Min, alpha*(Cf*sum(o[i] for i in 1:M) + 
Ct * sum(y[k, i, j] for k in 1:N, i in 1:M, j in 1:N) + 
Cw * sum(sum(y[k,i,j]*Ty[i,j] + z[k,j,i]*Tz[j,i] for i in 1:M, j in 1:N) + sum(x[k, j1, j2] * Tx[j1, j2] for j1 in 1:N, j2 in 1:N) for k in 1:N)) +
(1 - alpha) * L)


#MTZ Constraitns
for k in 1:N
    for j in 1:N
        @constraint(model, u[k, j] <= N)
        @constraint(model, u[k, j] >= 1)
    end
end

#(MTZ) Between Demand Locations
for j1 in 1:N #loc 1
    for j2 in 1:N #loc 2
        if j1 != j2
            for k in 1:N
                @constraint(model, u[k, j2] - u[k, j1] >= 1 - N*(1-x[k,j1,j2]))
                #@constraint(model, u[k, j1] - u[k, j2] + N*x[k,j1,j2] <= N - 1)
            end
        end
    end
end

# Optional additional constriant
for j in 1:N
    for k in 1:N
        @constraint(model, x[k,j,j] == 0)
    end
end
# end additional constraint

for k in 1:N
    @constraint(model, sum(y[k,i,j] for i in 1:M, j in 1:N) <= 1)
end

for k in 1:N
    @constraint(model, sum(x[k,j1,j2] for j1 in 1:N, j2 in 1:N) <= (N - 1) * sum(y[k,i,j] for i in 1:M, j in 1:N))
end

for i in 1:M
    @constraint(model, sum(y[k,i,j] for k in 1:N, j in 1:N) <= N * o[i])
end

for k in 1:N
    for i in 1:M
        @constraint(model, sum(z[k,j,i] for j in 1:N) == sum(y[k,i,j] for j in 1:N))
    end
end

#FLOW CONSTRAINTS
for j2 in 1:N 
        @constraint(model, sum(sum(y[k,i,j2] for i in 1:M) + sum(x[k,j1,j2] for j1 in 1:N) for k in 1:N) == 1)
end 

for j1 in 1:N 
        @constraint(model, sum(sum(x[k,j1,j2] for j2 in 1:N) + sum(z[k,j1,i] for i in 1:M) for k in 1:N) == 1)
end 

# Now time constraints
for k in 1:N 
    @constraint(model, f[k] == sum(y[k,i,j] * Ty[i,j] for i in 1:M, j in 1:N) + sum(x[k,j1,j2] * Tx[j1,j2] for j1 in 1:N, j2 in 1:N))
end

for k in 1:N 
    @constraint(model, f[k] <= L)
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-05
Set parameter TimeLimit to value 120


In [5]:
optimize!(model)

Set parameter TimeLimit to value 120
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 875 rows, 5001 columns and 20660 nonzeros
Model fingerprint: 0xdee46b82
Variable types: 31 continuous, 4970 integer (4970 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e-02, 9e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 180 rows and 25 columns
Presolve time: 0.02s
Presolved: 695 rows, 4976 columns, 17435 nonzeros
Variable types: 25 continuous, 4951 integer (4950 binary)
Found heuristic solution: objective 16.6600000
Found heuristic solution: objective 4.0800000

Root relaxation: objective 2.076000e+00, 253 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbe

In [6]:
try
    rm("results/output.h5")
catch
    1
end
h5write("results/output.h5", "factories", value.(o))
h5write("results/output.h5", "x_edges", value.(x))
h5write("results/output.h5", "y_edges", value.(y))
h5write("results/output.h5", "z_edges", value.(z))


In [7]:
print(size(value.(x)))
value.(o)

(5, 5, 5)

95-element Vector{Float64}:
  1.0
  0.0
  0.0
  0.0
  0.0
  0.0
  1.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
 -0.0
 -0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [8]:
value.(L)

2.5

In [9]:
value.(u[5, :])

5-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0